# Importações

In [1]:
import re
import spacy
import pickle
import os
import pandas as pd
import csv
import glob
import csv
from spacy.training import offsets_to_biluo_tags
from spacy.training import biluo_tags_to_spans
from spacy.training import offsets_to_biluo_tags, biluo_tags_to_offsets, biluo_tags_to_spans
import itertools

# importações para o bloco de código do treinamento do modelo de NER
import random
from spacy.util import minibatch, compounding
from pathlib import Path
from spacy.tokens import Doc
from spacy.training import Example
import warnings
import datetime as date

### Carregando Dados Rotulados Manualmente Para Treinamento do Modelo

Carrega os dados no formato .csv e cria um dataframe a partir deles.

In [5]:
dataSet = pd.read_csv("../DataSets/Dataset_Padrao_IOB_Versao4.csv", usecols=['Palavras', 'Rotulo', 'Sentenca', 'Inicio', 'Fim'])

Carrega a base utilizada para rotulação manual das categorias. Utilizada para extrair as setenças a que cada token rotulado pertence.

In [93]:
with open("/home/arthur/Documentos/IC/git/ProjetoMestrado/DataSets/Dados_Utilizados_Para_RotulacaoManual.txt", "r", encoding='utf-8') as file:
    textos = file.read().splitlines()
print("Quantidade de linhas da base:" , len(textos))

Quantidade de linhas da base: 14628


Código para converter os dados que estão no formato IOB para o formato do spaCy

In [7]:
sMarker = 0 #Marcador que referência a qual setença a entidade pertence, ou seja, serve pra pegar o texto da referencia dos tokens no dataset
s = textos[sMarker]
dataSetEntrada = []
entities = []
for index, row in dataSet.iterrows():
    
    if(row['Sentenca']-1 != sMarker): # Nova sentença
        if entities:
            dataSetEntrada.append((s, {"entities": [tuple(e) for e in entities]})) # Salva sentença anterior
        entities = [] # esvazia entidades
        sMarker = row['Sentenca']-1 # atualiza o marcador de sentença
        
        if (sMarker < len(textos)): # Limite de textos
            s = textos[sMarker]

    if(row['Rotulo'][0] == 'B'):
         entities.append([row['Inicio'], row['Fim'], row['Rotulo'][2:]])
    if(row['Rotulo'][0] == 'I'):
        if (entities): 
            entities[-1][1] = row['Fim']
        else:
            # print(index)
            entities.append([row['Inicio'], row['Fim'], row['Rotulo'][2:]])
    
    if index == dataSet.index[-1]: # Ultimo elemento
        if entities:
            dataSetEntrada.append((s, {"entities": [tuple(e) for e in entities]}))
        entities = [] 


print('Quantidade Sentenças Rotuladas:', len(dataSetEntrada))

Quantidade Sentenças Rotuladas: 11294


In [8]:
TRAIN_DATA = dataSetEntrada[0:899]
TEST_DATA = dataSetEntrada[900:999]
#for index, item in enumerate(TRAIN_DATA):
    #print(index)

In [29]:
TEST_DATA

[('A Cachaça Dom Bré Amburana é elaborada com requinte e arte, respeitando as tradições seculares de produção da Cachaça Mineira.',
  {'entities': [(10, 17, 'NOME_BEBIDA'), (18, 26, 'TIPO_MADEIRA')]}),
 ('Após a destilação, a Cachaça Dom Bré é armazenada em tonéis de Amburana, o que lhe confere aroma agradável e um sabor suave, especial e inigualável.',
  {'entities': [(29, 36, 'NOME_BEBIDA'),
    (53, 59, 'RECIPIENTE_ARMAZENAMENTO'),
    (63, 71, 'TIPO_MADEIRA')]}),
 ('Armazenada em tonéis de amburana por um longo tempo, selecionada e elaborada harmoniozamente, com notas amadeiradas, frutadas e florais, com sabor e aroma exclusivos de uma bebida singular.',
  {'entities': [(14, 20, 'RECIPIENTE_ARMAZENAMENTO'),
    (24, 32, 'TIPO_MADEIRA'),
    (98, 115, 'CARACTERISTICA_SENSORIAL_SABOR'),
    (117, 125, 'CARACTERISTICA_SENSORIAL_SABOR'),
    (128, 135, 'CARACTERISTICA_SENSORIAL_SABOR')]}),
 ('Origem: Guarani / MG Madeira: Amburana Envelhecimento: 3 anos Grad.',
  {'entities': [(8, 15, 

### Cross-validation do dataset para dados de treino e teste

https://www.kaggle.com/slavaz/spacy-ner-ensemble

In [106]:
from spacy.symbols import ORTH
def custom_tokenizer(nlp):
    infixes = (
        spacy.lang.char_classes.LIST_ELLIPSES
        + spacy.lang.char_classes.LIST_ICONS
        + ["\.", "\/", ":", "\(", "\)"]
        + [
            r"(?<=[0-9])[+\-\*^](?=[0-9-])",
            r"(?<=[\.])[\.](?=[\.])",
            r"(?<=[{a}{q}])\.(?=[{a}{q}])".format(
                a = spacy.lang.char_classes.ALPHA, q = spacy.lang.char_classes.CONCAT_QUOTES),
            r"(?<=[{a}]),(?=[{a}])".format(a=spacy.lang.char_classes.ALPHA),
            r"(?<=[{a}])(?:{h}|\/)(?=[{a}])".format(a=spacy.lang.char_classes.ALPHA, h=spacy.lang.char_classes.LIST_HYPHENS),
            r"(?<=[{a}])(?:{h}|\/)(?=[0-9])".format(a=spacy.lang.char_classes.ALPHA, h=spacy.lang.char_classes.LIST_HYPHENS),
            r"(?<=[0-9])(?:{h}|\/)(?=[{a}])".format(a=spacy.lang.char_classes.ALPHA, h=spacy.lang.char_classes.LIST_HYPHENS),
            r"(?<=[0-9])(?:{h}|\/|,|\.)(?=[0-9])".format(a=spacy.lang.char_classes.ALPHA, h=spacy.lang.char_classes.LIST_HYPHENS),
            r"(?<=[{a}0-9])[:<>=/](?=[{a}])".format(a=spacy.lang.char_classes.ALPHA),
            r"(?<=[{a}\"]),(?=[0-9{a}])".format(a=ALPHA),
        ]
    )
    list_quotes = ["\\'", '"', '`',
                     '‘', '´', '’', '‚', ',',
                     '„', '»', '«', '「', '」',
                     '『', '』', '（', '）', '〔',
                     '〕', '【', '】', '《', '》',
                     '〈', '〉']
    suffixes = (
        LIST_PUNCT
        + LIST_HYPHENS
        + list_quotes
        + LIST_ICONS
        + [ "\/", ":", "\(", "\)"]
        + ["'s", "'S", "’s", "’S", "—", "–"]
        + [
            r"(?<=[0-9])\+",
            # r"(?<=°[FfCcKk])\.",
            r"(?<=[0-9])(?:{c})".format(c=CURRENCY),
            # r"(?<=[0-9])(?:{u})".format(u=UNITS),
            r"(?<=[0-9a-z(?:{q})])\.".format(
            al=ALPHA_LOWER, q=CONCAT_QUOTES),
            r"(?<=[{au}])\.".format(au=ALPHA_UPPER),
        ] 
    )
    
    prefixes = (
        LIST_PUNCT
        + [ "\/", ":", "\(", "\)"]
        + LIST_HYPHENS
        + list_quotes
        + LIST_CURRENCY
        + LIST_ICONS
    )
    # print(prefixes)

    infix_re = spacy.util.compile_infix_regex(infixes)
    suffix_re = spacy.util.compile_suffix_regex(suffixes)
    prefix_re = spacy.util.compile_prefix_regex(prefixes)
    # suffix_re = re.compile(r'''\.[\]\)"']$''')
    custom = Tokenizer(nlp.vocab, prefix_search=prefix_re.search,
                                suffix_search=suffix_re.search,
                                infix_finditer=infix_re.finditer,
                                token_match=nlp.tokenizer.token_match,
                                rules=None)
    special_case = [{ORTH: "."}, {ORTH: "."}, {ORTH: "."}]        # Adding special case rule
    custom.add_special_case("...", special_case)
    return custom
custom_tokenizer(nlp)

In [107]:
# Alterando Tokenizador
from spacy.tokenizer import Tokenizer
test_text = 'Premiações : 1º Lugar na ... "Exposição",aa 190% a-a do Formiga-MG”,) da arte a$moderna/1922 1º Lugar no concurso de cachaça 1111 2003 de qualidade Hotel Sofitel Rio de Janeiro/2005 E destaque entre as 10 melhores cachaças envelhecida na Expocachaça São Paulo(Mercado Municipal) 2011.'
# test_text = "ANÁLISE DO CACHACIER: De cor amarelo-clara, possui uma mescla de aromas florais e adocicados."
# test_text = "Premiações : 1º Lugar na Exposição do Centenário,semana da arte moderna/1922 1º Lugar no concurso de cachaça de qualidade Hotel Sofitel Rio de Janeiro/2005 E destaque entre as 10 melhores cachaças envelhecida na Expocachaça São Paulo(Mercado Municipal) 2011."
# test_text = "PREÇO: R$ 35,00"
doc = nlp(test_text)

print("Default tokenized text",end=' : ')
for token in doc:
    print(token,end=', ')



nlp.tokenizer = custom_tokenizer(nlp)
doc = nlp(test_text)

print("\nAFTER tokenized text",end=' : ')
tok_exp = nlp.tokenizer.explain(test_text)

for t in tok_exp:
    print(t[1], "\t", t[0])


# with open("/home/arthur/Documentos/IC/git/ProjetoMestrado/DataSets/Dados_Utilizados_Para_RotulacaoManual.txt", "r", encoding='utf-8') as text_file:
#     textos = text_file.read().splitlines()
# print("Quantidade de linhas da base:", len(textos))
# for test_text in textos:
#     doc = nlp(test_text)
#     with open("tokensBySpacyFix.txt","a") as file:
#         for token in doc:
#                 print(token)
#                 file.write(f"{token}\n")


print("\n\n")
print()
# tok_exp = nlp.tokenizer.explain(test_text)
# for t in tok_exp:
#     print(t[1], "\t", t[0])

# print(list(nlp.Defaults.infixes))


# with open("/home/arthur/Documentos/IC/git/ProjetoMestrado/DataSets/Dados_Utilizados_Para_RotulacaoManual.txt", "r", encoding='utf-8') as text_file:
#     textos = text_file.read().splitlines()
# print("Quantidade de linhas da base:", len(textos))
# for test_text in textos:
#     tok_exp = nlp.tokenizer.explain(test_text)
# # for t in tok_exp:
# #     print(t[1], "\t", t[0])
#     with open("tokensBySpacy.txt","a") as file:
#         for t in tok_exp:
#             print(t[1], "\t", t[0])
#             file.write(f"{t[1]}\t{t[0]}\n")

Default tokenized text : Premiações, :, 1º, Lugar, na, ..., ", Exposição",aa, 190, %, a-a, do, Formiga-MG, ”, ,, ), da, arte, a$, moderna/1922, 1º, Lugar, no, concurso, de, cachaça, 1111, 2003, de, qualidade, Hotel, Sofitel, Rio, de, Janeiro/2005, E, destaque, entre, as, 10, melhores, cachaças, envelhecida, na, Expocachaça, São, Paulo(Mercado, Municipal, ), 2011, ., 
AFTER tokenized text : Premiações 	 TOKEN
: 	 PREFIX
1º 	 TOKEN
Lugar 	 TOKEN
na 	 TOKEN
. 	 SPECIAL-1
. 	 SPECIAL-2
. 	 SPECIAL-3
" 	 PREFIX
Exposição" 	 TOKEN
, 	 INFIX
aa 	 TOKEN
190% 	 TOKEN
a 	 TOKEN
- 	 INFIX
a 	 TOKEN
do 	 TOKEN
Formiga 	 TOKEN
- 	 INFIX
MG” 	 TOKEN
, 	 SUFFIX
) 	 SUFFIX
da 	 TOKEN
arte 	 TOKEN
a$moderna 	 TOKEN
/ 	 INFIX
1922 	 TOKEN
1º 	 TOKEN
Lugar 	 TOKEN
no 	 TOKEN
concurso 	 TOKEN
de 	 TOKEN
cachaça 	 TOKEN
1111 	 TOKEN
2003 	 TOKEN
de 	 TOKEN
qualidade 	 TOKEN
Hotel 	 TOKEN
Sofitel 	 TOKEN
Rio 	 TOKEN
de 	 TOKEN
Janeiro 	 TOKEN
/ 	 INFIX
2005 	 TOKEN
E 	 TOKEN
destaque 	 TOKEN
entre 	 TOKEN
a

In [ ]:
print(list(nlp.Defaults.infixes))

In [ ]:
print(list(infixes))
# print(list(nlp.p))

In [108]:
# Obtendo o componente do pipeline para trabalhar com reconhecimentos de entidades nomeadas
ner = nlp.get_pipe('ner')

# Adicionando as categorias de entidade nomeada ao pipeline 'ner'
ner.add_label('CARACTERISTICA_SENSORIAL_AROMA')
ner.add_label('CARACTERISTICA_SENSORIAL_CONSISTÊNCIA')
ner.add_label('CARACTERISTICA_SENSORIAL_SABOR')
ner.add_label('CARACTERISTICA_SENSORIAL_COR')
ner.add_label('RECIPIENTE_ARMAZENAMENTO')
ner.add_label('EQUIPAMENTO_DESTILACAO')
ner.add_label('CLASSIFICACAO_BEBIDA')
ner.add_label('TEMPO_ARMAZENAMENTO')
ner.add_label('GRADUACAO_ALCOOLICA')
ner.add_label('TIPO_MADEIRA')
ner.add_label('NOME_BEBIDA')
ner.add_label('VOLUME')
ner.add_label('NOME_LOCAL')
ner.add_label('NOME_ORGANIZACAO')
ner.add_label('NOME_PESSOA')
ner.add_label('PRECO')
ner.add_label('TEMPO')

1

In [101]:
#verificando as categorias de entidades nomeadas contidas no pipeline
print("Categorias de entidade que o modelo contém:", '\n\n', ner.label_data)

Categorias de entidade que o modelo contém: 

 {0: Counter(), 1: Counter({'LOC': 137478, 'PER': 65030, 'MISC': 38659, 'ORG': 25003, 'CARACTERISTICA_SENSORIAL_AROMA': -1, 'CARACTERISTICA_SENSORIAL_CONSISTÊNCIA': -2, 'CARACTERISTICA_SENSORIAL_SABOR': -3, 'CARACTERISTICA_SENSORIAL_COR': -4, 'RECIPIENTE_ARMAZENAMENTO': -5, 'EQUIPAMENTO_DESTILACAO': -6, 'CLASSIFICACAO_BEBIDA': -7, 'TEMPO_ARMAZENAMENTO': -8, 'GRADUACAO_ALCOOLICA': -9, 'TIPO_MADEIRA': -10, 'NOME_BEBIDA': -11, 'VOLUME': -12, 'NOME_LOCAL': -13, 'NOME_ORGANIZACAO': -14, 'NOME_PESSOA': -15, 'PRECO': -16, 'TEMPO': -17}), 2: Counter({'LOC': 137478, 'PER': 65030, 'MISC': 38659, 'ORG': 25003, 'CARACTERISTICA_SENSORIAL_AROMA': -1, 'CARACTERISTICA_SENSORIAL_CONSISTÊNCIA': -2, 'CARACTERISTICA_SENSORIAL_SABOR': -3, 'CARACTERISTICA_SENSORIAL_COR': -4, 'RECIPIENTE_ARMAZENAMENTO': -5, 'EQUIPAMENTO_DESTILACAO': -6, 'CLASSIFICACAO_BEBIDA': -7, 'TEMPO_ARMAZENAMENTO': -8, 'GRADUACAO_ALCOOLICA': -9, 'TIPO_MADEIRA': -10, 'NOME_BEBIDA': -11, 'VOLU

In [109]:
# Obter os nomes dos componentes que queremos desativar durante o treinamento

#pipe_exceptions = ["ner", "trf_wordpiecer", "trf_tok2vec"]
#unaffected_pipes = [pipe for pipe in nlp.pipe_names if pipe not in pipe_exceptions]
unaffected_pipes = [pipe for pipe in nlp.pipe_names if pipe != 'ner']

### Treinando o modelo de reconhecimento de entidades nomeadas

In [111]:
# iniciar o ciclo de treinamento, apenas treinando NER

epochs = 10
optimizer = nlp.create_optimizer()
#optimizer = nlp.resume_training()
#optimizer = nlp.begin_training()

with nlp.disable_pipes(*unaffected_pipes), warnings.catch_warnings():
    warnings.filterwarnings("once", category=UserWarning, module='spacy')
    print(date.datetime.now())
    sizes = compounding(4.0, 32.0, 1.001)
    
    # agrupe os exemplos usando o minibatc do spaCy
    for epoch in range(epochs):
        examples = TRAIN_DATA
        random.shuffle(examples)
        batches = minibatch(examples, size=sizes)
        losses = {}
        
        for batch in batches:
            texts, annotations = zip(*batch)
            example = []
            # Atualize o modelo com a iteração de cada texto
            for i in range(len(texts)):
                doc = nlp.make_doc(texts[i])
                example.append(Example.from_dict(doc, annotations[i]))
                # Update the model
                nlp.update(example, drop=0.35, losses=losses,sgd=optimizer)
        print("Losses ({}/{})" .format(epoch + 1, epochs), '-', date.datetime.now(), losses)
            

2022-03-21 20:55:56.431104


/home/arthur/.local/lib/python3.8/site-packages/spacy/training/iob_utils.py:141: UserWarning: [W030] Some entities could not be aligned in the text "PREMIOS: 2017 - 2º Lugar na 27ª Edição Expocachaça..." with entities "[(9, 13, 'TEMPO'), (53, 67, 'NOME_LOCAL'), (68, 70...". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.warn(


Losses (1/1) - 2022-03-21 21:11:45.572053 {'ner': 52821.746543047004}


## Testando o Modelo

In [22]:
# Exemplo using o display sentence involving original entities
spacy.displacy.render(nlp("A Bacuri é uma cachaça, tem um sabor tropical que vai aumentar o seu ânimo e envolvê-lo em uma onda de puro prazer. É armazenada em barris de jequitibá"), style="ent")

### Carregando Dados para Teste

In [104]:
dados_Teste = [('A Cachaça Serra Limpa Prata 355ml é uma bebida armazenada por seis meses em tonéis de inox. Robusta. É uma das únicas Cachaças Artesanais cem porcento orgânicas do Brasil. Contém 50% de graduação alcoolia',
  {'entities': [(76, 91, 'RECIPIENTE_ARMAZENAMENTO')]}),
  
  ('A Cachaça Minha Deusa Vale Verde Prata 1000ml é uma bebida armazenada por um ano em tonéis de grápia, o que realça o aroma de cana-de-açúcar. Leve,. De sabor suave e marcante, é perfeita para o preparo de drinks e coquetéis.',
  {'entities': [(84, 100, 'RECIPIENTE_ARMAZENAMENTO')]}),

  ('Cachaça Mineirinha 355ml é uma edição especial das Cachaças Dona Beja, de Perdizes, Minas Gerais. Armazenada por cinco anos em barris de jequitiba. Chama a atenção a embalagem da bebida, que é envasada em garrafas de louça que imitam a cana-de-açúcar.',
  {'entities': [(127, 147, 'RECIPIENTE_ARMAZENAMENTO')]}),
 
  ('Três Corações é uma bebida armazenada por 2 anos em jequitibá. Robusta e com aroma doce', 
  {'entities':[(52, 61, 'RECIPIENTE_ARMAZENAMENTO')]})          
    ]

### Avaliação do Modelo

In [25]:
# Compara item por item o que foi classificado pelo modelo e a classifcação correta(manual)

#nlp = spacy.load(output_dir)
examples = []

for text, annots in TEST_DATA:
    print(f'Manual Anotation: {annots}')
    doc = nlp(text)
    #doc = nlp.make_doc(text)
    #pacy.displacy.render(nlp(text))
    
    for item in doc.ents:
        print('Model prediction: ', 'entities: ', [(item.start_char, item.end_char,item.label_)])
                
    examples.append(Example.from_dict(doc, annots)) 
    #print('Tokens', [(t.text, t.ent_type_, t.ent_iob) for t in doc])

Manual Anotation: {'entities': [(10, 17, 'NOME_BEBIDA'), (18, 26, 'TIPO_MADEIRA')]}
Model prediction:  entities:  [(10, 17, 'NOME_BEBIDA')]
Model prediction:  entities:  [(18, 26, 'TIPO_MADEIRA')]
Manual Anotation: {'entities': [(29, 36, 'NOME_BEBIDA'), (53, 59, 'RECIPIENTE_ARMAZENAMENTO'), (63, 71, 'TIPO_MADEIRA')]}
Model prediction:  entities:  [(29, 36, 'NOME_BEBIDA')]
Model prediction:  entities:  [(53, 59, 'RECIPIENTE_ARMAZENAMENTO')]
Model prediction:  entities:  [(63, 71, 'TIPO_MADEIRA')]
Model prediction:  entities:  [(118, 123, 'CARACTERISTICA_SENSORIAL_SABOR')]
Model prediction:  entities:  [(136, 147, 'CARACTERISTICA_SENSORIAL_SABOR')]
Manual Anotation: {'entities': [(14, 20, 'RECIPIENTE_ARMAZENAMENTO'), (24, 32, 'TIPO_MADEIRA'), (98, 115, 'CARACTERISTICA_SENSORIAL_SABOR'), (117, 125, 'CARACTERISTICA_SENSORIAL_SABOR'), (128, 135, 'CARACTERISTICA_SENSORIAL_SABOR')]}
Model prediction:  entities:  [(14, 20, 'RECIPIENTE_ARMAZENAMENTO')]
Model prediction:  entities:  [(24, 32, 'T

Model prediction:  entities:  [(7, 15, 'PRECO')]
Manual Anotation: {'entities': [(8, 13, 'VOLUME')]}
Model prediction:  entities:  [(8, 13, 'VOLUME')]
Manual Anotation: {'entities': [(25, 31, 'NOME_BEBIDA'), (32, 37, 'CLASSIFICACAO_BEBIDA'), (38, 43, 'VOLUME')]}
Model prediction:  entities:  [(25, 31, 'NOME_BEBIDA')]
Model prediction:  entities:  [(32, 37, 'CLASSIFICACAO_BEBIDA')]
Model prediction:  entities:  [(38, 43, 'VOLUME')]
Manual Anotation: {'entities': [(7, 15, 'PRECO')]}
Model prediction:  entities:  [(7, 15, 'PRECO')]
Manual Anotation: {'entities': [(30, 36, 'NOME_BEBIDA'), (37, 42, 'CLASSIFICACAO_BEBIDA'), (43, 48, 'VOLUME'), (65, 71, 'RECIPIENTE_ARMAZENAMENTO'), (75, 83, 'TIPO_MADEIRA')]}
Model prediction:  entities:  [(30, 36, 'NOME_BEBIDA')]
Model prediction:  entities:  [(37, 42, 'CLASSIFICACAO_BEBIDA')]
Model prediction:  entities:  [(43, 48, 'VOLUME')]
Model prediction:  entities:  [(65, 71, 'RECIPIENTE_ARMAZENAMENTO')]
Model prediction:  entities:  [(75, 83, 'TIPO_MA

Model prediction:  entities:  [(21, 40, 'CARACTERISTICA_SENSORIAL_COR')]
Model prediction:  entities:  [(47, 61, 'CARACTERISTICA_SENSORIAL_AROMA')]
Model prediction:  entities:  [(72, 87, 'CARACTERISTICA_SENSORIAL_SABOR')]
Model prediction:  entities:  [(93, 126, 'CARACTERISTICA_SENSORIAL_AROMA')]
Model prediction:  entities:  [(137, 149, 'CARACTERISTICA_SENSORIAL_SABOR')]
Manual Anotation: {'entities': [(8, 16, 'TIPO_MADEIRA'), (34, 44, 'CARACTERISTICA_SENSORIAL_SABOR'), (66, 86, 'CARACTERISTICA_SENSORIAL_SABOR'), (89, 98, 'CARACTERISTICA_SENSORIAL_SABOR'), (195, 202, 'CARACTERISTICA_SENSORIAL_SABOR')]}
Model prediction:  entities:  [(8, 16, 'TIPO_MADEIRA')]
Model prediction:  entities:  [(34, 44, 'CARACTERISTICA_SENSORIAL_SABOR')]
Model prediction:  entities:  [(66, 86, 'CARACTERISTICA_SENSORIAL_AROMA')]
Model prediction:  entities:  [(89, 98, 'CARACTERISTICA_SENSORIAL_AROMA')]
Model prediction:  entities:  [(195, 202, 'CARACTERISTICA_SENSORIAL_AROMA')]
Manual Anotation: {'entities':

### Métricas Precision, Recall e f-measure: 

In [26]:
# Apresenta o resultados das métricas para o modelo como um todo, e as métricas do modelo para cada categoria de entidade

print(nlp.evaluate(examples))

{'token_acc': 1.0, 'token_p': 1.0, 'token_r': 1.0, 'token_f': 1.0, 'pos_acc': None, 'morph_acc': None, 'morph_per_feat': None, 'sents_p': None, 'sents_r': None, 'sents_f': None, 'dep_uas': None, 'dep_las': None, 'dep_las_per_type': None, 'tag_acc': None, 'lemma_acc': None, 'ents_p': 0.8558951965065502, 'ents_r': 0.8032786885245902, 'ents_f': 0.828752642706131, 'ents_per_type': {'NOME_BEBIDA': {'p': 0.8333333333333334, 'r': 0.8333333333333334, 'f': 0.8333333333333334}, 'TIPO_MADEIRA': {'p': 1.0, 'r': 0.9545454545454546, 'f': 0.9767441860465117}, 'RECIPIENTE_ARMAZENAMENTO': {'p': 0.9090909090909091, 'r': 0.9090909090909091, 'f': 0.9090909090909091}, 'CARACTERISTICA_SENSORIAL_SABOR': {'p': 0.3076923076923077, 'r': 0.2222222222222222, 'f': 0.2580645161290323}, 'CARACTERISTICA_SENSORIAL_AROMA': {'p': 0.42105263157894735, 'r': 0.42105263157894735, 'f': 0.42105263157894735}, 'NOME_LOCAL': {'p': 1.0, 'r': 0.8695652173913043, 'f': 0.9302325581395349}, 'TEMPO_ARMAZENAMENTO': {'p': 1.0, 'r': 0.92

### Avaliação do Modelo pelas Métrica Acuracia

In [27]:
# dictionary which will be populated with the entities and result information
entity_evaluation = {}

# helper function to udpate the entity_evaluation dictionary
def update_results(entity, metric):
    if entity not in entity_evaluation:
        entity_evaluation[entity] = {"correct": 0, "total": 0}
    
    entity_evaluation[entity][metric] += 1

# same as before, see if entities from test set match what spaCy currently predicts
for data in TEST_DATA:
    sentence = data[0]
    entities = data[1]["entities"]

    for entity in entities:
        doc = nlp(sentence)
        correct_text = sentence[entity[0]:entity[1]]

        for ent in doc.ents:
            if ent.label_ == entity[2] and ent.text == correct_text:
                update_results(ent.label_, "correct")
                break

        update_results(entity[2], "total")
print("Resultado para cada categoria:",'\n', entity_evaluation)

Resultado para cada categoria: 
 {'NOME_BEBIDA': {'correct': 25, 'total': 30}, 'TIPO_MADEIRA': {'correct': 21, 'total': 22}, 'RECIPIENTE_ARMAZENAMENTO': {'correct': 10, 'total': 11}, 'CARACTERISTICA_SENSORIAL_SABOR': {'correct': 4, 'total': 18}, 'NOME_LOCAL': {'correct': 20, 'total': 23}, 'TEMPO_ARMAZENAMENTO': {'correct': 13, 'total': 14}, 'GRADUACAO_ALCOOLICA': {'correct': 8, 'total': 8}, 'VOLUME': {'correct': 37, 'total': 37}, 'PRECO': {'correct': 15, 'total': 15}, 'CARACTERISTICA_SENSORIAL_AROMA': {'correct': 8, 'total': 19}, 'CARACTERISTICA_SENSORIAL_COR': {'correct': 4, 'total': 7}, 'CARACTERISTICA_SENSORIAL_CONSISTÊNCIA': {'correct': 1, 'total': 4}, 'CLASSIFICACAO_BEBIDA': {'correct': 16, 'total': 18}, 'NOME_ORGANIZACAO': {'correct': 3, 'total': 5}, 'TEMPO': {'correct': 7, 'total': 9}, 'EQUIPAMENTO_DESTILACAO': {'correct': 1, 'total': 1}, 'NOME_PESSOA': {'correct': 3, 'total': 3}}


In [28]:
sum_total = 0
sum_correct = 0

for entity in entity_evaluation:
    total = entity_evaluation[entity]["total"]
    correct = entity_evaluation[entity]["correct"]

    sum_total += total
    sum_correct += correct
    
    print("{} | {:.2f}%".format(entity, correct / total * 100))

print()
print("Overall accuracy: {:.2f}%".format(sum_correct / sum_total * 100))

NOME_BEBIDA | 83.33%
TIPO_MADEIRA | 95.45%
RECIPIENTE_ARMAZENAMENTO | 90.91%
CARACTERISTICA_SENSORIAL_SABOR | 22.22%
NOME_LOCAL | 86.96%
TEMPO_ARMAZENAMENTO | 92.86%
GRADUACAO_ALCOOLICA | 100.00%
VOLUME | 100.00%
PRECO | 100.00%
CARACTERISTICA_SENSORIAL_AROMA | 42.11%
CARACTERISTICA_SENSORIAL_COR | 57.14%
CARACTERISTICA_SENSORIAL_CONSISTÊNCIA | 25.00%
CLASSIFICACAO_BEBIDA | 88.89%
NOME_ORGANIZACAO | 60.00%
TEMPO | 77.78%
EQUIPAMENTO_DESTILACAO | 100.00%
NOME_PESSOA | 100.00%

Overall accuracy: 80.33%


## Salvando o Modelo

In [ ]:
# Save the  model to directory

from pathlib import Path
output_dir=Path('C:\\Users\\prisc\\Downloads\\Modelo_spaCy\\model_saved')
nlp.to_disk(output_dir)
print("Saved model to", output_dir)

## Importando o Modelo

In [ ]:
# Load the saved model and predict

print("Loading from", output_dir)
nlp_updated = spacy.load(output_dir)
doc = nlp_updated("Eu gostei da Cachaça Artesanal criada em Minas Gerais" )
print("Entidades", [(ent.text, ent.label_) for ent in doc.ents])